# Data Preprocessing

In [67]:
import pandas as pd
import operator

#read_videos_file
videos = pd.read_excel('videos.xlsx')

In [68]:
import operator
def find_highest_confidence_object(video_features_from_ads):
    split_first = video_features_from_ads.split("|")
    get_objects = [x.split(":")[0] for x in split_first]
    get_confidence = [int(x.split(":")[1]) for x in split_first]
    converted_dict = dict(zip(get_objects, get_confidence))
    return max(converted_dict.items(), key=operator.itemgetter(1))[0]
videos['ad_cat'] = videos['detected_objects_withconfidence'].apply(find_highest_confidence_object)

In [69]:
videos.head(2)

,video_id,detected_objects_withconfidence,ad_cat
0,46,Shirt:20|Trousers:20|Footwear:20|Handbag:20|Wa...,Shirt
1,47,Shirt:0|Trousers:0|Footwear:0|Handbag:0|Watch:...,Shirt


In [70]:
video_mapper = pd.Series(videos.ad_cat.values,index=videos.video_id).to_dict()

In [71]:
watch_history = pd.read_excel('watch_history_final.xlsx')
watch_history.head(2)

,id,user_id,watched_video_id,watch_time_in_sec,ad_cat,ad_cat_id,click,ad_id
0,1,202,9195,74,Headphones,8,0,146
1,2,333,7331,143,Headphones,8,0,146


In [72]:
users = pd.read_excel('user_profile.xlsx')
users.head(2)

,user_id,final_gender_code,age_level,occupation
0,1,1,5,3
1,2,1,4,9


In [73]:
users[users['user_id']==155].final_gender_code

154    1
Name: final_gender_code, dtype: int64

In [74]:
ad_category = pd.read_excel('ad_cateogy.xlsx')
ad_category.head(2)

,ad_cat_id,category_name,adtitle
0,1,Shirt,Ad for Shirt
1,2,Trousers,Ad for Trousers


In [75]:
ad_category_mapper = pd.Series(ad_category.ad_cat_id.values,index=ad_category.category_name)

In [76]:
ad_category_mapper.get('Shirt')

1

In [77]:
ads = pd.read_excel('ads.xlsx')
ads.head(2)

,ad_id,ad_cat_id,brand_id,ad_title
0,1,1,16,hI56FpLEGb
1,2,2,8,OrZ6WLgGZY


In [78]:
combined = pd.merge(watch_history,users,how='left',on='user_id')
combined.head(2)

,id,user_id,watched_video_id,watch_time_in_sec,ad_cat,ad_cat_id,click,ad_id,final_gender_code,age_level,occupation
0,1,202,9195,74,Headphones,8,0,146,2,4,10
1,2,333,7331,143,Headphones,8,0,146,2,2,8


In [79]:
combined_v2 = pd.merge(combined,ads.drop('ad_cat_id',axis=1),how='left',on='ad_id')
combined_v2.head(2)

,id,user_id,watched_video_id,watch_time_in_sec,ad_cat,ad_cat_id,click,ad_id,final_gender_code,age_level,occupation,brand_id,ad_title
0,1,202,9195,74,Headphones,8,0,146,2,4,10,29,H0ijXu4iKz
1,2,333,7331,143,Headphones,8,0,146,2,2,8,29,H0ijXu4iKz


In [80]:
combined_v2 = combined_v2.drop('ad_cat',axis=1)

In [81]:
# refrence_data = combined_v2.to_dict('records')
# refrence_data[1]

# Model

In [82]:
import pandas as pd
import tensorflow as tf
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import logging, os 
logging.disable(logging.WARNING) 

from deepctr.estimator import DeepFMEstimator
from deepctr.estimator.inputs import input_fn_pandas

In [83]:
df = combined_v2.copy()
data = df.copy()

In [84]:
df.head(2)

,id,user_id,watched_video_id,watch_time_in_sec,ad_cat_id,click,ad_id,final_gender_code,age_level,occupation,brand_id,ad_title
0,1,202,9195,74,8,0,146,2,4,10,29,H0ijXu4iKz
1,2,333,7331,143,8,0,146,2,2,8,29,H0ijXu4iKz


In [85]:
sparse_features = ['user_id', 'watched_video_id', 'ad_cat_id','ad_id', 'final_gender_code', 'age_level', 'occupation',
       'brand_id', 'ad_title']
dense_features = ['watch_time_in_sec']
target = ['click']

In [86]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [87]:
# 2.count #unique features for each sparse field,and record dense feature field name

dnn_feature_columns = []
linear_feature_columns = []


In [88]:
for i, feat in enumerate(sparse_features):
    dnn_feature_columns.append(tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_identity(feat, data[feat].max() + 1), 4))
    linear_feature_columns.append(tf.feature_column.categorical_column_with_identity(feat, data[feat].max() + 1))
for feat in dense_features:
    dnn_feature_columns.append(tf.feature_column.numeric_column(feat))
    linear_feature_columns.append(tf.feature_column.numeric_column(feat))


In [89]:
# 3.generate input data for model

train, test = train_test_split(data, test_size=0.2, random_state=2021)

In [90]:
# Not setting default value for continuous feature. filled with mean.

train_model_input = input_fn_pandas(train, sparse_features + dense_features, 'click', shuffle=True)
test_model_input = input_fn_pandas(test, sparse_features + dense_features, None, shuffle=False)

In [91]:
# 4.Define Model,train,predict and evaluate
model = DeepFMEstimator(linear_feature_columns, dnn_feature_columns, task='binary',
                        config=tf.estimator.RunConfig(tf_random_seed=2021))


In [92]:
model.train(train_model_input)
pred_ans_iter = model.predict(test_model_input)
pred_ans = list(map(lambda x: x['pred'], pred_ans_iter))

C:\Users\dahal\.conda\envs\3.6new\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [93]:
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

test LogLoss 0.9209
test AUC 0.4964


# Prediction

In [94]:
ads = pd.read_excel('ads.xlsx')
ads.head(2)

,ad_id,ad_cat_id,brand_id,ad_title
0,1,1,16,hI56FpLEGb
1,2,2,8,OrZ6WLgGZY


In [95]:
# ##Test users here
# ## Creating test case here. 


# test = df.iloc[70:71].drop('ad_id',axis=1)
# test = test.reset_index(drop=True)
# test = test.drop('click',axis=1) ## Click data not yet received
# test = test.drop('brand_id',axis=1)
# test = test.drop('ad_title',axis=1)
# test.at[0,'watch_time_in_sec'] = np.array(0) ## watch_time data not yet received
# test

In [96]:
user_id_mapper = dict(set(zip(df.user_id,data.user_id)))
video_id_mapper = dict(set(zip(df.watched_video_id,data.watched_video_id)))

ad_category_mapper_v2 = dict(set(zip(df.ad_cat_id,data.ad_cat_id)))

ad_id_mapper = dict(set(zip(df.ad_id,data.ad_id)))
gender_code_mapper = dict(set(zip(df.final_gender_code,data.final_gender_code)))
age_level_mapper = dict(set(zip(df.age_level,data.age_level)))
occupation_mapper = dict(set(zip(df.occupation,data.occupation)))
brand_id_mapper = dict(set(zip(df.brand_id,data.brand_id)))
ad_title_mapper = dict(set(zip(df.ad_title,data.ad_title)))

# ad_id	final_gender_code	age_level	occupation	brand_id	ad_title

# Select UserID to test

In [97]:
#Inputs user_id and video_id

user_id = 155
video_id  = 9195

In [98]:
data.columns

Index(['id', 'user_id', 'watched_video_id', 'watch_time_in_sec', 'ad_cat_id',
       'click', 'ad_id', 'final_gender_code', 'age_level', 'occupation',
       'brand_id', 'ad_title'],
      dtype='object')

In [99]:
n_user_id = user_id_mapper.get(user_id)
watched_video_id = video_id_mapper.get(video_id)
watch_time_in_sec = 0 #Not received data yet
ad_cat = video_mapper.get(watched_video_id)
ad_cat_id = ad_category_mapper.get(ad_cat)
ad_cat_id_v2 = ad_category_mapper_v2.get(ad_cat_id)


final_gender_code = gender_code_mapper.get(list(users[users['user_id']==user_id].final_gender_code)[0])
age_level = age_level_mapper.get(list(users[users['user_id']==user_id].age_level)[0])
occupation = occupation_mapper.get(list(users[users['user_id']==user_id].occupation)[0])




In [100]:
new_df = ads[ads['ad_cat_id']==ad_cat_id]
new_df = new_df.reset_index(drop=True)
new_df.head(2)

,ad_id,ad_cat_id,brand_id,ad_title
0,7,6,22,rDdpdYoN6j
1,25,6,25,IgLtUcyeoU


In [101]:
new_df['ad_id']=new_df['ad_id'].apply(lambda x: ad_id_mapper.get(x))

In [102]:
new_df['ad_cat_id']=new_df['ad_cat_id'].apply(lambda x: ad_category_mapper_v2.get(x))

In [103]:
new_df['brand_id']=new_df['brand_id'].apply(lambda x: brand_id_mapper.get(x))

In [104]:
new_df['ad_title']=new_df['ad_title'].apply(lambda x: ad_title_mapper.get(x))

In [105]:
new_df.head()

,ad_id,ad_cat_id,brand_id,ad_title
0,6,5,21,341
1,24,5,24,118
2,31,5,27,386
3,47,5,13,170
4,55,5,17,189


In [106]:
import numpy as np

In [107]:
new_df['user_id'] = np.array(n_user_id,dtype='int64')
new_df['watched_video_id'] = np.array(watched_video_id,dtype='int64')
new_df['watch_time_in_sec'] = np.array(watch_time_in_sec,dtype='int64')
new_df['ad_cat_id'] = np.array(ad_cat_id_v2,dtype='int64')

new_df['final_gender_code'] = np.array(final_gender_code,dtype='int64')
new_df['age_level'] = np.array(age_level,dtype='int64')

new_df['occupation'] = np.array(occupation,dtype='int64')


In [108]:
new_df = new_df[['user_id','watched_video_id','watch_time_in_sec','ad_cat_id','ad_id','final_gender_code','age_level','occupation','brand_id','ad_title']]

In [109]:
new_df.head()

,user_id,watched_video_id,watch_time_in_sec,ad_cat_id,ad_id,final_gender_code,age_level,occupation,brand_id,ad_title
0,154,5798,0,5,6,0,1,0,21,341
1,154,5798,0,5,24,0,1,0,24,118
2,154,5798,0,5,31,0,1,0,27,386
3,154,5798,0,5,47,0,1,0,13,170
4,154,5798,0,5,55,0,1,0,17,189


In [110]:
# def mapper(dictionary,key):
#     return dictionary.get(key)

In [111]:
# ads.head()

In [112]:
# new_df = pd.DataFrame({'user_id':n_user_id, 'watched_video_id':watched_video_id, 
#                        'watch_time_in_sec':watch_time_in_sec, 'ad_cat_id':ad_cat})

In [113]:
# selection = df.iloc[70:71].drop('ad_id',axis=1)
# index = df.iloc[70:71].drop('ad_id',axis=1).index.to_list()
# selection

In [114]:
# data.iloc[index].drop('ad_id',axis=1)

In [115]:
# ads = ads[ads.ad_cat_id==test['ad_cat_id'][0]]
# ads = ads.reset_index(drop=True)
# test = test.merge(ads,how='left',on='ad_cat_id')
# test.head(2)

In [116]:
# predict = predict[['user_id','watched_video_id','watch_time_in_sec','ad_cat_id','ad_id','final_gender_code','age_level','occupation','brand_id','ad_title']]

In [117]:
# import numpy as np
# test['ad_id'] = ''
# ads = ads[ads.ad_cat_id==2]
# ads = ads.reset_index(drop=True)
# test.at[0,'ad_id'] = np.array(ads.ad_id.to_list())

In [118]:
# predict = test.explode('ad_id')
# # predict['ad_id']=predict['ad_id'].astype('int64')
# predict = predict.reset_index(drop=True)
# predict.head()

In [119]:
# predict = predict[['user_id','watched_video_id','watch_time_in_sec','ad_cat_id','ad_id','final_gender_code','age_level','occupation','brand_id','ad_title']]

In [120]:
# predict['ad_id']=predict['ad_id'].astype('int64')

In [121]:
# new = predict.copy()

In [122]:
# sparse_features = ['user_id', 'watched_video_id', 'ad_cat_id','ad_id', 'final_gender_code', 'age_level', 'occupation',
#        'brand_id', 'ad_title']
# dense_features = ['watch_time_in_sec']
# # target = ['click']

In [123]:
# for feat in sparse_features:
# #     lbe = LabelEncoder()
#     new[feat] = lbe.fit_transform(new[feat])
# # mms = MinMaxScaler(feature_range=(0, 1))
# new[dense_features] = mms.fit_transform(new[dense_features])

In [124]:
predict_model_input = input_fn_pandas(new_df, sparse_features + dense_features, None, shuffle=False)

In [125]:
pred_ans_iter = model.predict(predict_model_input)
pred_ans = list(map(lambda x: x['pred'], pred_ans_iter))

C:\Users\dahal\.conda\envs\3.6new\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [126]:
index = [i for i,value in enumerate(pred_ans) if value>=0.9]
if len(index)==0:
    index = [i for i,value in enumerate(pred_ans) if value>=0.8]

In [127]:
final = new_df.iloc[index]
final

,user_id,watched_video_id,watch_time_in_sec,ad_cat_id,ad_id,final_gender_code,age_level,occupation,brand_id,ad_title
2,154,5798,0,5,31,0,1,0,27,386
36,154,5798,0,5,349,0,1,0,27,21


# Ads to Display with highest probablity of Click

In [128]:
#Select first Only
# final[['ad_cat_id','ad_id','brand_id','ad_title']]

In [129]:
final_ad_ids = [x for x, y in ad_id_mapper.items() if y in final.ad_id.to_list()]
print(f'final ad_ids to display: {final_ad_ids}')

final ad_ids to display: [350, 32]


In [130]:
ads[ads.ad_id.isin(final_ad_ids)]

,ad_id,ad_cat_id,brand_id,ad_title
31,32,6,28,x57lptT636
349,350,6,28,2a1ScswGKr
